# imports

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.io as pio

pio.templates.default = 'plotly_white'
pd.set_option('display.float_format', '{:.5f}'.format)

# read data

In [3]:
df = pd.read_csv("../data/interim/loans_int.csv")

/home/dangj/PycharmProjects/fca-takehome-test/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.



# Data cleaning

 - Remove highly correlated variables 
 - To decide which variable to remove check other correlation with other variables
 
 - proceed to train model via back/fwrd selection or recusive feature selection
 - make sure methods are motivated vs other methods

In [4]:
df.columns

Index(['account_id', 'installment', 'loan_amount', 'interest_rate', 'term',
       'purpose', 'issue_date', 'description', 'title', 'home_ownership',
       'annual_income', 'employment_length', 'job_title',
       'earliest_credit_line', 'public_records', 'last_record_months',
       'last_delinquency_months', 'last_derog_months', 'delinquency_2y',
       'inquiries_6m', 'open_accounts', 'debt_to_income', 'credit_card_usage',
       'credit_card_balance', 'total_current_balance', 'nr_accounts',
       'loan_status', 'amount_payed', 'year', 'district', 'postcode_district',
       'credit_score'],
      dtype='object')

In [7]:
df.dtypes

account_id                   int64
installment                float64
loan_amount                float64
interest_rate              float64
term                        object
purpose                     object
issue_date                  object
description                 object
title                       object
home_ownership              object
annual_income               object
employment_length           object
job_title                   object
earliest_credit_line        object
public_records             float64
last_record_months         float64
last_delinquency_months    float64
last_derog_months          float64
delinquency_2y             float64
inquiries_6m               float64
open_accounts              float64
debt_to_income             float64
credit_card_usage          float64
credit_card_balance        float64
total_current_balance      float64
nr_accounts                float64
loan_status                 object
amount_payed               float64
year                

In [6]:
df.head(5)

,account_id,installment,loan_amount,interest_rate,term,purpose,issue_date,description,title,home_ownership,...,credit_card_usage,credit_card_balance,total_current_balance,nr_accounts,loan_status,amount_payed,year,district,postcode_district,credit_score
0,125968,829.10000,25000.00000,11.89000,36 months,debt_consolidation,Aug-2009,Due to a lack of personal finance education an...,Debt consolidation for on-time payer,RENT,...,52.10000,28854.00000,nan,42.00000,Fully Paid,29324.32000,2009,Liverpool,L69,710.69972
1,128479,40.50000,1200.00000,13.11000,36 months,debt_consolidation,Mar-2010,"If funded, I would use this loan consolidate t...",zxcvb,OWN,...,40.40000,2584.00000,nan,31.00000,Fully Paid,1457.31000,2010,Redbridge,IG9,416.08693
2,128650,366.86000,10800.00000,13.57000,36 months,debt_consolidation,Nov-2009,I currently have a personal loan with Citifina...,Nicolechr1978,RENT,...,25.60000,3511.00000,nan,40.00000,Fully Paid,13195.27000,2009,Rugby,CV21,354.22883
3,129758,264.11000,7200.00000,19.05000,36 months,debt_consolidation,Aug-2012,"Credit cards are out of here, I am tired of be...",caminijio,RENT,...,90.10000,3874.00000,154930.00000,25.00000,Fully Paid,9100.80466,2012,West Lindsey,DN21,697.21416
4,130240,102.92000,3000.00000,14.26000,36 months,credit_card,Sep-2009,I am seeking to refinance a credit account whi...,Rejecting new cardmember agreement,MORTGAGE,...,39.50000,4740.00000,nan,23.00000,Fully Paid,3703.38000,2009,South Derbyshire,DE11,799.42870


In [8]:
df.nunique()

account_id                 237436
installment                 43308
loan_amount                  1328
interest_rate                 413
term                            2
purpose                        14
issue_date                     84
description                 77364
title                       42797
home_ownership                  6
annual_income               18837
employment_length              11
job_title                  123190
earliest_credit_line          645
public_records                 14
last_record_months            122
last_delinquency_months       123
last_derog_months             151
delinquency_2y                 24
inquiries_6m                    9
open_accounts                  59
debt_to_income               3976
credit_card_usage            1199
credit_card_balance         46736
total_current_balance      131405
nr_accounts                   105
loan_status                     5
amount_payed               227572
year                            7
district      